In [1]:
import subprocess
import os

In [2]:
# Suppress native-hadoop warning
!sed -i '$a\# Add the line for suppressing the NativeCodeLoader warning \nlog4j.logger.org.apache.hadoop.util.NativeCodeLoader=ERROR,console' /$HADOOP_HOME/etc/hadoop/log4j.properties

In [3]:
def load_to_hdfs(user, path):
    if user == '':
        print('User not specified. For shared directory, use "hadoop" as user.')
        return

    # Get the current working directory
    base_dir = os.getcwd()
    data_dir = os.path.join(base_dir, 'work', 'data')

    dataset_dirs = [os.path.join(data_dir, 'train'), os.path.join(data_dir, 'test')]
    if path == 'train':
        dataset_dirs = [os.path.join(data_dir, 'train')]
    if path == 'test':
        dataset_dirs = [os.path.join(data_dir, 'test')]

    # Load dir to hdfs
    for path in dataset_dirs:
        subprocess.run(['hdfs', 'dfs', '-mkdir', '-p', f'/user/{user}/data'])
        subprocess.run(['hdfs', 'dfs', '-put', path, f'/user/{user}/data'])

In [4]:
# users = ['hadoop', 'blake', 'thanya', 'jennifer']
users = ['hadoop']
for user in users:
    load_to_hdfs(user, 'train')
    load_to_hdfs(user, 'test')

In [5]:
# list directories in hdfs for all users
for user in users:
    print(f'\nListing directories for user: {user}')
    subprocess.run(['hdfs', 'dfs', '-ls', f'/user/{user}/data/train'])


Listing directories for user: hadoop
Found 10 items
-rw-r--r--   1 root supergroup 1116948159 2024-05-03 05:10 /user/hadoop/data/train/train_0.txt
-rw-r--r--   1 root supergroup 1165261648 2024-05-03 05:10 /user/hadoop/data/train/train_1.txt
-rw-r--r--   1 root supergroup 1167807043 2024-05-03 05:10 /user/hadoop/data/train/train_2.txt
-rw-r--r--   1 root supergroup 1168046738 2024-05-03 05:10 /user/hadoop/data/train/train_3.txt
-rw-r--r--   1 root supergroup 1167900398 2024-05-03 05:10 /user/hadoop/data/train/train_4.txt
-rw-r--r--   1 root supergroup 1243252130 2024-05-03 05:10 /user/hadoop/data/train/train_5.txt
-rw-r--r--   1 root supergroup 1241075737 2024-05-03 05:10 /user/hadoop/data/train/train_6.txt
-rw-r--r--   1 root supergroup 1241481728 2024-05-03 05:10 /user/hadoop/data/train/train_7.txt
-rw-r--r--   1 root supergroup 1241984479 2024-05-03 05:10 /user/hadoop/data/train/train_8.txt
-rw-r--r--   1 root supergroup  143179231 2024-05-03 05:10 /user/hadoop/data/train/train_9.t